In [ ]:
import pandas as pd
import numpy as np
import gutenbergpy.textget
import re
import string
from nltk import ngrams
from nltk.tokenize import word_tokenize, sent_tokenize

In [ ]:
book_authors = {
    'Charles Dickens': [46, 98, 1400, 730, 766, 1023, 786, 580, 883],
    'Herman Melville': [2701, 11231, 15859, 21816, 34970, 10712, 12384, 4045, 8118, 2694, 13720, 53861],
    'Jane Austen': [1342, 158, 161, 105, 121, 141, 946]
}

In [ ]:
book_contents = {}
for book_id in [book_id for id_list in book_authors.values() for book_id in id_list]:
    # Load in the book
    raw_book = gutenbergpy.textget.get_text_by_id(book_id)
    clean_book = gutenbergpy.textget.strip_headers(raw_book)
    # Convert to string
    book = clean_book.decode('UTF-8')
    # Remove first characters of the book (remove chapter headings, etc)
    #book = book[10000:]
    book_contents[book_id] = book


In [21]:
books_wtoks = {}
books_stoks = {}
for book_id, book in book_contents.items():
    books_wtoks[book_id] = word_tokenize(book)
    books_stoks[book_id] = sent_tokenize(book)

In [33]:
book_stats = {}
for book_id in book_contents.keys():
    book_stats[book_id] = {}
    book_stok = books_stoks[book_id]
    book_wtok = books_wtoks[book_id]
    (book_stats[book_id])['sent_chars'] = [len(sent) for sent in book_stok]
    (book_stats[book_id])['sent_words'] = [len(word_tokenize(sent)) for sent in book_stok]
    (book_stats[book_id])['word_chars'] = [len(word) for word in book_wtok]
    book_stats[book_id]['']
    print(np.mean(sent_words))
    print(np.median(sent_words))
    print(np.std(sent_words))
    print("--")

46
18.32853470437018
12.0
16.97572842138022
--
98
29.67627178939879
24.0
23.17328660637072
--
1400
31.41277128547579
27.0
22.904760070545574
--
730
30.48169057064388
24.0
25.032162109863553
--
766
29.669275013376136
24.0
22.403857317331944
--
1023
27.705058717253838
22.0
22.428735772942527
--
786
29.39525330899133
24.0
23.527145344618948
--
580
22.55559558117195
15.0
21.433893128530872
--
883
29.918962133876008
23.0
24.728649689621182
--
2701
28.242083195409908
22.0
25.610818646013584
--
11231
24.566523605150216
20.0
19.467505542181065
--
15859
31.49157581764123
24.0
50.61455991428356
--
21816
23.740896358543417
16.0
22.800505199925098
--
34970
28.1742586365026
21.0
26.10967199611641
--
10712
29.708668453976767
26.0
23.107199105599225
--
12384
26.063143445435827
22.0
20.835700560503344
--
4045
27.881183541377716
26.0
16.393345517008008
--
8118
36.80811518324607
32.0
26.5224904484633
--
2694
31.263157894736842
24.0
27.934490907252638
--
13720
24.739696775571215
22.0
18.916122951899347
-

In [30]:
np.mean(sent_words)

21.409794075597887

In [ ]:
grams = list(ngrams(book_contents[98], 2))

In [ ]:
sorted(grams)[0:10]

In [ ]:
list(grams)
new_grams = grams.copy()
gram_count = {}
for i in range(0, len(grams)-1):
    gram = new_grams.pop(0)
    if gram in new_grams:
        if gram not in gram_count:
            gram_count[gram] = 0
        gram_count[gram] += 1

In [ ]:
dict(sorted(gram_count.items(), key=lambda item: -item[1]))